In [1]:
# notebook import

from ipynb.fs.full.data_processing import *

well_fpath_list: <class 'list'> 
 ['/Users/sarahfisher/Documents/project/data/20230404/CSV_FB_20230404/Time_points_dfs_01.csv', '/Users/sarahfisher/Documents/project/data/20230404/CSV_FB_20230404/Time_points_dfs_03.csv', '/Users/sarahfisher/Documents/project/data/20230404/CSV_FB_20230404/Time_points_dfs_04.csv', '/Users/sarahfisher/Documents/project/data/20230404/CSV_FB_20230404/Time_points_dfs_05.csv', '/Users/sarahfisher/Documents/project/data/20230404/CSV_FB_20230404/Time_points_dfs_06.csv', '/Users/sarahfisher/Documents/project/data/20230404/CSV_FB_20230404/Time_points_dfs_07.csv', '/Users/sarahfisher/Documents/project/data/20230404/CSV_FB_20230404/Time_points_dfs_09.csv', '/Users/sarahfisher/Documents/project/data/20230404/CSV_FB_20230404/Time_points_dfs_10.csv', '/Users/sarahfisher/Documents/project/data/20230404/CSV_FB_20230404/Time_points_dfs_11.csv', '/Users/sarahfisher/Documents/project/data/20230404/CSV_FB_20230404/Time_points_dfs_12.csv', '/Users/sarahfisher/Documents/proje

feature_df: <class 'pandas.core.frame.DataFrame'> 
                 gfp         rfp  area
0        528.690625  237.327083   960
1        403.250916  216.184982   546
2        384.769737  210.777961  1216
3        346.063130  202.326790  1885
4        330.310096  198.775240   832
...             ...         ...   ...
223548   901.914583  262.665972  1440
223549  1032.986538  276.790385   520
223550   921.374693  263.346437   814
223551   938.197815  267.918919  1739
223552   981.538084  263.349713  1221

[2720208 rows x 3 columns]
ncells_list_dict: <class 'dict'> 
 {1: [56, 52, 63, 39, 42, 72, 112, 121, 147, 118, 28, 39, 35], 2: [41, 72, 46, 42, 54, 83, 98, 133, 130, 105, 33, 80, 57], 3: [56, 63, 38, 36, 42, 64, 116, 116, 138, 98, 63, 72, 63], 4: [32, 73, 35, 33, 37, 66, 99, 107, 127, 105, 41, 73, 48], 5: [66, 50, 27, 46, 52, 78, 101, 114, 131, 98, 52, 88, 61], 6: [42, 62, 46, 35, 31, 82, 114, 113, 108, 120, 41, 77, 69], 7: [47, 82, 34, 40, 39, 71, 86, 114, 143, 89, 45, 68, 67], 8: [59,

bin_dict: <class 'dict'> 
 {'gfp': array([ 226.76648352,  311.67626067,  396.58603782,  481.49581497,
        566.40559213,  651.31536928,  736.22514643,  821.13492358,
        906.04470074,  990.95447789, 1075.86425504, 1160.77403219,
       1245.68380935, 1330.5935865 , 1415.50336365, 1500.4131408 ,
       1585.32291796, 1670.23269511, 1755.14247226, 1840.05224941,
       1924.96202657, 2009.87180372, 2094.78158087, 2179.69135802]), 'rfp': array([ 169.59401709,  223.45627941,  277.31854173,  331.18080405,
        385.04306637,  438.90532869,  492.76759101,  546.62985332,
        600.49211564,  654.35437796,  708.21664028,  762.0789026 ,
        815.94116492,  869.80342723,  923.66568955,  977.52795187,
       1031.39021419, 1085.25247651, 1139.11473883, 1192.97700115,
       1246.83926346, 1300.70152578, 1354.5637881 , 1408.42605042]), 'area': array([500.0, 565.2173913043479, 630.4347826086956, 695.6521739130435,
       760.8695652173913, 826.0869565217391, 891.304347826087,
       9

In [ ]:
# user input parameters

# boolean parameter to dictate whether code is run
run = False

# boolean parameter to dictate use of print statements
debug = False

# boolean parameter to dictate whether results are written to .csv files
write = False

In [9]:
# input: timestep and reference feature name of interest (timestep, feature_ref)
# output: dictionary mapping a well's id number to its list of values of a given feature at an individual timestep
def get_timestep_population_dict(timestep, feature_ref):
    population_dict = {key:[] for key in timestep_list}
    for well in experiment_dict:
        well_df = experiment_dict[well]
        timestep_vals = list((well_df.loc[well_df[timestep_ref] == timestep])[feature_ref])
        population_dict[well] = timestep_vals
    return population_dict

# input: list of measurements (population) and a bin configuration (bins)
# output: shannon entropy for a given population and number of bins
def shannon_entropy_fn(population, bins):
    hist, _ = np.histogram(population, bins=bins, density=True)
    hist = hist[hist > 0]
    return entropy(hist)

# input: dictionary of populations (population_dict), size of random joint sampling of the populations (subset_size), and bin configuration (bins)
# output: mutual information at a timestep 
def calculate_mi(population_dict, subset_size, bins):
    try:
        joint_pop = np.concatenate([np.random.choice(population_dict[pop], subset_size) for pop in population_dict])
        joint_shannon_entropy = shannon_entropy_fn(joint_pop, bins)

        shannon_entropy_list = []
        for _, pop in enumerate(population_dict):
            shannon_entropy = shannon_entropy_fn(population_dict[pop], bins)
            shannon_entropy_list.append(shannon_entropy)
        mean_entropy = sum(shannon_entropy_list) / len(shannon_entropy_list)
        
        mi = joint_shannon_entropy - mean_entropy

        return mi
    except:
        print("Error calculating MI.")
        
        return None
    
# input: reference feature name of interst (feature_ref), bin configuration (bins), and an optional .csv output file path (csv_fpath)
# output: list of mutual information measurements over time for given feature
def get_mi_list(feature_ref, bins, csv_fpath=None):
    mi_list = []
    for timestep in timestep_list:
        population_dict = get_timestep_population_dict(timestep, feature_ref)
        subset_size = int(ncells_avg_dict[timestep]//len(well_id_list))
        mi = calculate_mi(population_dict, subset_size, bins)
        mi_list.append(mi)
        if csv_fpath != None:
            append_row_csv(csv_fpath, [mi])
    return mi_list

In [13]:
# `main()` function declaration

# intended use in coordination with `mi_calculation_plot.ipynb`
# requires: import of `data_processing.ipynb` and user input parameters specified above
# output: None (assigns new global variables `bin_edges_dict`, `mi_list_dict` and writes to output .csv files)

def main():
    
    # dictionary mapping each feature to its optimal bin edge configuration
    global bin_edges_dict
    bin_edges_dict = create_bin_edges_dict()
    
    # dictionary mapping each feature to a list of its mutual information over time
    global mi_list_dict
    mi_list_dict = {}
    
    for (name, ref) in zip(feature_name_list, feature_ref_list):
        
        # get list of mutual information values
        if write:
            csv_fpath = results_fpath + mi_calc_ext + f'{name}_mi_list.csv'
            append_row(csv_fpath, ['mi'])
            mi_list = get_mi_list(ref, bin_edges_dict[name], csv_fpath)
        
        else:
            mi_list = get_mi_list(ref, bin_edges_dict[name])
            
        mi_list_dict[name] = mi_list
        
    if debug:
        print('mi_list_dict:', type(mi_list_dict), '\n', mi_list_dict)
        
    return

In [ ]:
# call to `main()` function

if run:
    main()

In [11]:
print(mi_list_dict)

{'gfp': [0.4594241459350856, 0.4176423389986619, 0.38065248348306535, 0.3918359901541144, 0.4181957434780492, 0.40011128730369605, 0.44215746898171293, 0.3965115781699371, 0.380819427970714, 0.4089721949714291, 0.37067264151464285, 0.27306825635323895, 0.26319544042627374, 0.3285488145293416, 0.3132506658025884, 0.3276279920741989, 0.3319985668693495, 0.2565611518732491, 0.25576162691863336, 0.2529426044236087, 0.2525736099696496, 0.14577272438722133, 0.17317555028327458, 0.1659633533644771, 0.16924101148410298, 0.1349090316331234, 0.14136474234815655, 0.11165495150386007, 0.0738670658499947, 0.10716293537202337, 0.11006585665438973, 0.07432805216137028, 0.11011839240000354, 0.12448767772069358, 0.056446406545576755, 0.08892705379071564, 0.06492697553216331, 0.034998498203486106, 0.06820893142312756, 0.08214261259493538, 0.06773018463871727, 0.09524418485625863, 0.10241058542891879, 0.07136203121854012, 0.1209053815210206, 0.08478124904129891, 0.10059243454027356, 0.09556209067570065, 